# Cyclical epidemic dynamics
Both in models and in reality, we can observe regular cycles in
the epidemiology of infectious disease outbreaks.


In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_sir_model(model_config):

    compartments = (
        "susceptible", 
        "latent", 
        "infectious", 
        "recovered",
    )
    model = CompartmentalModel(
        times=(
            model_config["start_time"],
            model_config["end_time"],
        ),
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution={
            "susceptible": model_config["population"] - model_config["seed"], 
            "infectious": model_config["seed"],
        }
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    return model

In [ ]:
model_config = {
    "start_time": 0.,
    "end_time": 1e3,
    "population": 1e5,
    "seed": 1.,
}
sir_wane_model = build_sir_model(model_config)

sir_wane_model.add_transition_flow(
    name="waning", 
    fractional_rate=Parameter("replenishment"), 
    source="recovered", 
    dest="susceptible",
)

In [ ]:
# Raw input parameters
parameters = {
    "recovery_rate": 0.333,
    "contact_rate": 1.,
    "replenishment": 0.005,
}
population = 1e5
seed = 1.

In [ ]:
sir_demog_model = build_sir_model(model_config)

sir_demog_model.add_universal_death_flows(
    "universal_death",
    death_rate=Parameter("replenishment"),
)
sir_demog_model.add_replacement_birth_flow(
    "births",
    "susceptible",
)

In [ ]:
sir_wane_model.run(parameters=parameters)
outputs = sir_wane_model.get_outputs_df()
outputs.plot()

In [ ]:
sir_demog_model.run(parameters=parameters)
sir_demog_model.get_outputs_df().plot()

## Phase plane

In [ ]:
outputs_after_100 = outputs.loc[outputs.index > 70.]

In [ ]:
fig = px.line(
    outputs_after_100, 
    x="susceptible", 
    y="infectious"
)
fig.show()

In [ ]:
px.line_3d(
    outputs_after_100, 
    x="susceptible", 
    y="infectious", 
    z=outputs_after_100.index
)